<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/Gemma_3N_4B_finetuning_with_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Competition](attachment:4ba23f21-a6df-45a7-b3ab-0460b9700288.png)](https://www.kaggle.com/competitions/google-gemma-3n-hackathon/overview)

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Kaggle instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).

## Gemma 3N Notebook collection:
1. Gemma 3N **Multimodal inference + conversational finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-multimodal-finetuning-inference) *⬅️ Your are here*
2. Gemma 3N **Vision finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-vision-finetuning)
3. Gemma 3N **Audio finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-audio-finetuning)

### Installation

In [8]:
!nvidia-smi

Fri Nov 21 12:51:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
    !pip install weave
    !pip install rouge_score

In [ ]:
#%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade transformers # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [15]:
from huggingface_hub import notebook_login

notebook_login()

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision, Text and Audio models!

In [1]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "google/gemma-3n-E2B-it", # "google/gemma-3n-E4B-it" or "unsloth/gemma-3n-E4B-it", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 768, # 1024 Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Gemma3N patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.co

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Gemma 3N can process Text, Vision and Audio!

Let's first experience how Gemma 3N can handle multimodal inputs. We use Gemma 3N's recommended settings of `temperature = 1.0, top_p = 0.95, top_k = 64`

In [16]:
from transformers import TextStreamer
import gc
# Helper function for inference
def do_gemma_3n_inference(model, messages, max_new_tokens = 128):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # Must add for generation
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")
    _ = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    # Cleanup to reduce VRAM usage
    del inputs
    torch.cuda.empty_cache()
    gc.collect()

# Gemma 3N can see images!

<img src="https://files.worldwildlife.org/wwfcmsprod/images/Sloth_Sitting_iStock_3_12_2014/story_full_width/8l7pbjmj29_iStock_000011145477Large_mini__1_.jpg" alt="Alt text" height="256">

In [ ]:
sloth_link = "https://files.worldwildlife.org/wwfcmsprod/images/Sloth_Sitting_iStock_3_12_2014/story_full_width/8l7pbjmj29_iStock_000011145477Large_mini__1_.jpg"

messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : sloth_link },
        { "type": "text",  "text" : "Mba'e pelíkulapepa osë ko mymba?" }
    ]
}]
# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

Let's make a poem about sloths!

In [ ]:
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : "Ehai peteĩ ñe'ẽpoty perezoso (peteï mymba ojoguáva Mapinguari-pe) peguara: " }]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 128)

# Gemma 3N can also hear!

In [ ]:
from IPython.display import Audio, display
Audio("https://www.nasa.gov/wp-content/uploads/2015/01/591240main_JFKmoonspeech.mp3")

In [ ]:
!wget -qqq https://www.nasa.gov/wp-content/uploads/2015/01/591240main_JFKmoonspeech.mp3 -O audio.mp3

In [ ]:
audio_file = "audio.mp3"

messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "text",  "text" : "Mba'érehepa oñe'ẽ ko audio?" }
    ]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

# Let's combine all 3 modalities together!

In [ ]:
messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "image", "image" : sloth_link },
        { "type": "text",  "text" : "Mba'érehepa oñe'ẽ ko audio ha ta'anga? "\
                                    "Mba'éichapa ojoaju hikuái?" }
    ]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

# Let's finetune Gemma 3N!

You can finetune the vision and text parts for now through selection - the audio part can also be finetuned - we're working to make it selectable as well!

We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 4,           # 4-8-16..., Larger = higher accuracy, but might overfit
    lora_alpha = 4,  # 4-8-16..., Recommended alpha == r at least
    lora_dropout = 0, # increase if the model overfit
    bias = "none", # or lora-only
    random_state = 42, #3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

We get the first 3000 rows of the dataset

In [5]:
from datasets import load_dataset
#dataset = load_dataset("mlabonne/FineTome-100k", split = "train[:3000]")
dataset = load_dataset("rubuntu/dataset-guarani-jopara-v01")

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [6]:
from unsloth.chat_templates import standardize_data_formats
dataset_train = standardize_data_formats(dataset["train"])

In [7]:
dataset_dev = standardize_data_formats(dataset["validation"])

In [8]:
dataset_test = standardize_data_formats(dataset["test"])

Let's see how row 100 looks like!

In [9]:
dataset_train[100]

{'instruction': "Diseña peteî base de datos oregistra haguã mba'apohára sueldo.",
 'input': '',
 'output': "Pe base de datos oguerekova’erã umi campo mba’apohara réra, cargo, sueldo ha ára rehegua. Avei oikeva'erã peteî campo mba'apohára sambyhyhárape guarã, ikatu haguã umi sueldo oñemboja'o hekoitépe departamento-kuéra rupi. Avei base de datos ikatuva'erã ogenera informe gasto salarial rehegua umi departamento térã tapichakuérape guarã."}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [10]:
def formatting_prompts_func(examples):
   convos_1 = examples["instruction"]
   convos_2 = examples["input"]
   convos_3 = examples["output"]

   texts = []
   for i in range(len(convos_1)):
       # Ensure content parts are strings, replacing None with empty string
       instruction_part = str(convos_1[i]) if convos_1[i] is not None else ""
       input_part = str(convos_2[i]) if convos_2[i] is not None else ""
       output_part = str(convos_3[i]) if convos_3[i] is not None else ""

       user_content = instruction_part
       if input_part: # Only add input if it's not empty
           user_content += " " + input_part

       # Apply chat template with robust content
       full_text = tokenizer.apply_chat_template(
           [
               {'role': 'user', 'content': [{'type': 'text', 'text': user_content}]},
               {'role': 'assistant', 'content': [{'type': 'text', 'text': output_part}]},
           ],
           tokenize = False,
           add_generation_prompt = False
       ).removeprefix('<bos>')
       texts.append(full_text)
   return { "text" : texts, }

dataset_train = dataset_train.map(formatting_prompts_func, batched = True)
dataset_dev = dataset_dev.map(formatting_prompts_func, batched = True)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True)

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [11]:
dataset_train[100]["text"]

"<start_of_turn>user\nDiseña peteî base de datos oregistra haguã mba'apohára sueldo.<end_of_turn>\n<start_of_turn>model\nPe base de datos oguerekova’erã umi campo mba’apohara réra, cargo, sueldo ha ára rehegua. Avei oikeva'erã peteî campo mba'apohára sambyhyhárape guarã, ikatu haguã umi sueldo oñemboja'o hekoitépe departamento-kuéra rupi. Avei base de datos ikatuva'erã ogenera informe gasto salarial rehegua umi departamento térã tapichakuérape guarã.<end_of_turn>\n"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [12]:
! pip install evaluate bert_score

In [12]:
from transformers import EvalPrediction
import numpy as np
import evaluate
# For METEOR and BLEU, NLTK data is required
import nltk
# Ensure punkt and wordnet are downloaded for tokenization and stemming required by METEOR/BLEU
try:
    nltk.data.find('tokenizers/punkt')
except Exception as e:
    print(e)
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/wordnet')
except Exception as e:
    print(e)
    nltk.download('wordnet', quiet=True)


# Load metrics outside the function to avoid re-loading repeatedly
# bertscore requires internet access the first time it runs to download a model (e.g., 'bert-base-uncased')
try:
    bertscore_metric = evaluate.load("bertscore")
except Exception as e:
    print(f"Could not load BERTScore. Ensure internet is available for initial download: {e}")
    bertscore_metric = None

try:
    meteor_metric = evaluate.load("meteor")
except Exception as e:
    print(f"Could not load METEOR. Ensure nltk data is downloaded: {e}")
    meteor_metric = None

try:
    rouge_metric = evaluate.load("rouge")
except Exception as e:
    print(f"Could not load ROUGE. Ensure internet is available for initial download: {e}")
    rouge_metric = None

# Note: The tokenizer must be accessible in this scope for the decode function to work.
# Assuming 'tokenizer' is defined globally where this function runs.

########################
# Disabled for RAM/GPU #
########################
# None = disabled
bertscore_metric = None
meteor_metric = None
#rouge_metric = None
########################

def compute_metrics(p: EvalPrediction):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    pred_ids = np.argmax(logits, axis=-1)
    label_ids = p.label_ids

    decoded_preds = []
    decoded_labels = []

    for i in range(len(label_ids)):
        # For references (labels):
        actual_label_tokens = label_ids[i][label_ids[i] != -100]
        if len(actual_label_tokens) > 0:
            decoded_label = tokenizer.decode(actual_label_tokens, skip_special_tokens=True)
            decoded_labels.append(decoded_label)
        else:
            decoded_labels.append("")

        # For predictions:
        response_start_idx = np.where(label_ids[i] != -100)
        if len(response_start_idx) > 0 and len(response_start_idx[0]) > 0:
            response_start_idx = response_start_idx[0][0]
            predicted_response_tokens = pred_ids[i][response_start_idx:]
            decoded_pred = tokenizer.decode(predicted_response_tokens, skip_special_tokens=True)
            decoded_preds.append(decoded_pred)
        else:
            decoded_preds.append("")

    # Filter out empty strings which can cause issues with metric computation
    filtered_decoded_labels = [text for text in decoded_labels if text.strip()]
    filtered_decoded_preds = [text for text in decoded_preds if text.strip()]

    # Handle cases where there are no valid samples for metric computation
    if not filtered_decoded_labels or not filtered_decoded_preds:
        return {
            "meteor": 0.0,
            "bertscore_f1": 0.0,
            "rouge_f1": 0.0
        }

    metrics = {}

    # METEOR (Requires NLTK downloads, faster than BERTScore, better than ROUGE/BLEU)
    if meteor_metric:
        meteor_results = meteor_metric.compute(predictions=filtered_decoded_preds,
                                              references=filtered_decoded_labels)
        metrics["meteor"] = meteor_results["meteor"]

    # BERTScore (Slower due to loading/running a BERT model for embeddings, best semantic metric here)
    if bertscore_metric:
        # Lang parameter is crucial. If text is English, 'en' is default.
        # Ensure you handle different languages if necessary.
        bert_results = bertscore_metric.compute(predictions=filtered_decoded_preds,
                                                references=filtered_decoded_labels,
                                                model_type="microsoft/deberta-base-mnli",
                                                #lang="en"
                                                )
        # BERTScore returns precision, recall, and f1 for each sample. We average the F1 scores.
        metrics["bertscore_f1"] = np.mean(bert_results["f1"])

    # ROUGE (Lightweight metric)
    if rouge_metric:
        rouge_results = rouge_metric.compute(predictions=filtered_decoded_preds,
                                             references=filtered_decoded_labels)
        # Using rougeL for overall summary, can use rouge1 or rouge2 as well
        metrics["rouge_f1"] = rouge_results["rougeL"].mid.fmeasure

    # Combined metric calculation
    combined_score_sum = 0.0
    active_metric_count = 0

    if "meteor" in metrics:
        combined_score_sum += metrics["meteor"]
        active_metric_count += 1
    if "bertscore_f1" in metrics:
        combined_score_sum += metrics["bertscore_f1"]
        active_metric_count += 1
    if "rouge_f1" in metrics:
        combined_score_sum += metrics["rouge_f1"]
        active_metric_count += 1

    metrics["combined"] = combined_score_sum / active_metric_count if active_metric_count > 0 else 0.0

    return metrics


**********************************************************************
  Resource wordnet not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('wordnet')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/wordnet

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Could not load ROUGE. Ensure internet is available for initial download: To be able to use evaluate-metric/rouge, you need to install the following dependencies['rouge_score'] using 'pip install rouge_score' for instance'


In [13]:
from transformers import TrainerCallback
import psutil
import os
import torch

# Define the MemoryMonitorCallback class as provided by the user
class MemoryMonitorCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        # Python process RAM usage
        process = psutil.Process(os.getpid())
        python_ram_mb = process.memory_info().rss / (1024 ** 2)

        # GPU VRAM usage
        if torch.cuda.is_available():
            gpu_allocated_mb = torch.cuda.memory_allocated() / (1024 ** 2)
            gpu_max_allocated_mb = torch.cuda.max_memory_allocated() / (1024 ** 2)
            print(f"\nStep {state.global_step}: Python RAM = {python_ram_mb:.2f} MB, GPU Allocated = {gpu_allocated_mb:.2f} MB, GPU Max Allocated = {gpu_max_allocated_mb:.2f} MB")
        else:
            print(f"\nStep {state.global_step}: Python RAM = {python_ram_mb:.2f} MB")

# Instantiate the callback
memory_monitor_callback = MemoryMonitorCallback()

In [35]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_dev, # Can set up evaluation!
    #compute_metrics = compute_metrics, # Now includes ROUGE and BLEU!
    callbacks=[memory_monitor_callback], # Add the custom callback here
    #
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8, # Use GA to mimic batch size!
        warmup_steps = 5,
        #num_train_epochs = 1, # 3, Set this for full training run.
        max_steps = 900, # 500
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 300, # 250
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        report_to = "wandb", # Use this for WandB etc
        do_eval=True,
        eval_strategy="steps", # epoch if num_train_epochs
        eval_steps=300, # 250
        #metric_for_best_model="combined",
        #greater_is_better=True, # loss -> lo contrario
        gradient_checkpointing = True, # New: Enable gradient checkpointing
        remove_unused_columns=True, # for test
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [36]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [37]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>user\nDiseña peteî base de datos oregistra haguã mba'apohára sueldo.<end_of_turn>\n<start_of_turn>model\nPe base de datos oguerekova’erã umi campo mba’apohara réra, cargo, sueldo ha ára rehegua. Avei oikeva'erã peteî campo mba'apohára sambyhyhárape guarã, ikatu haguã umi sueldo oñemboja'o hekoitépe departamento-kuéra rupi. Avei base de datos ikatuva'erã ogenera informe gasto salarial rehegua umi departamento térã tapichakuérape guarã.<end_of_turn>\n"

Now let's print the masked out example - you should see only the answer is present:

In [38]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                               Pe base de datos oguerekova’erã umi campo mba’apohara réra, cargo, sueldo ha ára rehegua. Avei oikeva'erã peteî campo mba'apohára sambyhyhárape guarã, ikatu haguã umi sueldo oñemboja'o hekoitépe departamento-kuéra rupi. Avei base de datos ikatuva'erã ogenera informe gasto salarial rehegua umi departamento térã tapichakuérape guarã.<end_of_turn>\n"

In [39]:
# @title Show current memory stats
import gc
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.379 GB of memory reserved.


# Let's train the model!

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1 | Total steps = 900
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 5,283,840 of 5,444,722,112 (0.10% trained)


Step,Training Loss,Validation Loss
300,2.845700,2.496565
600,2.357000,2.385138


Unsloth: Not an error, but Gemma3nForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient



Step 300: Python RAM = 3934.38 MB, GPU Allocated = 7843.74 MB, GPU Max Allocated = 11611.63 MB

Step 300: Python RAM = 3964.27 MB, GPU Allocated = 7843.74 MB, GPU Max Allocated = 12583.86 MB

Step 600: Python RAM = 3812.69 MB, GPU Allocated = 7843.74 MB, GPU Max Allocated = 12583.86 MB

Step 600: Python RAM = 3813.08 MB, GPU Allocated = 7843.74 MB, GPU Max Allocated = 12583.86 MB


In [ ]:
def tokenize_and_prepare_for_eval(examples):
    # Tokenize the 'text' field which was created by formatting_prompts_func
    tokenized_inputs = tokenizer(
        text=examples["text"],
        max_length=1024, # Use the model's max sequence length (set during model loading)
        truncation=True,
        padding='max_length', # Pad to max_length to ensure uniform tensor shapes for batched processing
        return_attention_mask=True,
        return_tensors='np', # Ensure NumPy arrays are returned
    )
    # Create labels, initially a copy of input_ids. The data collator will then mask the instruction part.
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Apply the tokenization and label creation to dataset_test
dataset_test_prepared = dataset_test.map(
    tokenize_and_prepare_for_eval,
    batched=True,
    remove_columns=dataset_test.column_names, # Remove original columns like 'text', 'input', 'instruction', 'output'
)

print("Prepared dataset_test columns:", dataset_test_prepared.column_names)
print("First entry of prepared dataset_test:", dataset_test_prepared[0])

Now that `dataset_test` has been properly tokenized and includes the `input_ids`, `attention_mask`, and `labels` columns, you can evaluate the model using the `trainer`:

In [ ]:
# Now evaluate the model with the prepared dataset
trainer.evaluate(dataset_test_prepared)

In [ ]:
# @title Show final memory and time stats
gc.collect()
torch.cuda.empty_cache()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Ehaíve ko secuencia: 1, 1, 2, 3, 5, 8,", # fibonacci
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Mba'érepa hovy yvága?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
#model.save_pretrained("gemma-3n")  # Local saving
#tokenizer.save_pretrained("gemma-3n")
model.push_to_hub("mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it") #, token = hf_token) # Online saving
tokenizer.push_to_hub("mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it") #, token = hf_token) # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it", #"lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Mba'épa ha'e pe Gemma-3N?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3N-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3N-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if True: # Change to True to upload finetune
    try:
        trainer.push_to_hub("mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it")
    except Exception as e:
        print(e)
        model.push_to_hub_merged(
            "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it", tokenizer,
            #token = hf_token
        )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if True: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it-gguf", # This is the repo_id for the GGUF model
        tokenizer = tokenizer,
        #quantization_type = "Q8_0", # Removed as it's not a valid argument for push_to_hub_gguf
        #token = hf_token,
    )

Now, use the `gemma-3N-finetune.gguf` file or `gemma-3N-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

## Gemma 3N Notebook collection:
1. Gemma 3N **Multimodal inference + conversational finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-multimodal-finetuning-inference) *⬅️ Your are here*
2. Gemma 3N **Vision finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-vision-finetuning)
3. Gemma 3N **Audio finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-audio-finetuning)

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
